In [1]:
import torch
import numpy as np
import random

#Certifying ResNet_4B requires large GPU memory, I would suggest moving to the CPU
device = 'cuda:0' if torch.cuda.is_available else 'cpu'

In [2]:
from auto_LiRPA import BoundedModule, PerturbationLpNorm, BoundedTensor
from auto_LiRPA.utils import get_spec_matrix
from cert_util import min_correct_with_eps, load_data, DeltaWrapper

from model_defs import resnet2b, resnet4b

In [3]:
cifar10_mean = (0.4914, 0.4822, 0.4465)  # np.mean(train_set.train_data, axis=(0,1,2))/255
cifar10_std = (0.2471, 0.2435, 0.2616)  # np.std(train_set.train_data, axis=(0,1,2))/255

mu = torch.tensor(cifar10_mean).view(3,1,1).to(device)
std = torch.tensor(cifar10_std).view(3,1,1).to(device)

def normalize(X, device):
    result = (X.to(device) - mu.to(device))/std.to(device)
    return result

def bounded_results(eps,bounded_model):
    # normalization for the input noise
    ptb = PerturbationLpNorm(x_L=-eps/std.view(1,3,1,1).expand(1,3,32,32), x_U=eps/std.view(1,3,1,1).expand(1,3,32,32))
    bounded_delta = BoundedTensor(delta, ptb)
    
    final_name = bounded_model.final_name
    input_name = '/1' # '/input.1' 

    result = bounded_model.compute_bounds(
        x=(new_image,bounded_delta), method='backward', C=C,
        return_A=True, 
        needed_A_dict={ final_name: [input_name] },
    )
    lower, upper, A_dict = result
    lA = A_dict[final_name][input_name]['lA']
    uA = A_dict[final_name][input_name]['uA']

    lb = lower - ptb.concretize(delta, lA, sign=-1)
    ub = upper - ptb.concretize(delta, uA, sign=1)


    lA = torch.reshape(lA,(eval_num, num_cls-1,-1))
    return lA,lb, lower

### Reproducibility

In [4]:
my_seed = 0
torch.cuda.empty_cache()
torch.manual_seed(my_seed)
random.seed(my_seed)
np.random.seed(my_seed)

### Hyperparameters 

In [5]:
eval_num = 100
num_cls = 10
adv_e = 1

# CIFAR models, one can choose from 'resnet2b', 'resnet4b'
model_name = "resnet2b" 
adv_train_norm = 8

### Loadding the model for certification

In [6]:
net = eval(model_name)()
net.load_state_dict(torch.load('./model_weights/'
                                +model_name+ '_' +
                                 str(adv_train_norm) +
                                '.pth', map_location=device)['state_dict'])

<All keys matched successfully>

### Loading a batch of data

In [7]:
new_image, new_label = load_data(num_imgs=eval_num, random=True, dataset='CIFAR')
new_image = normalize(new_image,device)
C = get_spec_matrix(new_image,new_label.long(),10)

Files already downloaded and verified


### Model concretization

In [8]:
eps = adv_e/255
delta = torch.zeros_like(new_image[0]).unsqueeze(0)
dummy_input = (new_image, delta)
model = DeltaWrapper(net.to(device))
bounded_model = BoundedModule(model, dummy_input)
bounded_model.eval()
final_name = bounded_model.final_name

### Results and comparision 

In [9]:
print('Eps:', adv_e)

alpha,beta,result = bounded_results(eps,bounded_model)
samp_ACC = torch.sum(result.detach().cpu().min(axis=1)[0] > 0).numpy()
print('Samp-wise Cert-ACC: {}%'.format(samp_ACC))

label = new_label
number_class = num_cls
cert_ACC, delta = min_correct_with_eps(alpha, beta, eps, label, number_class=10, verbose=False, dataset='CIFAR')
print('UP-based Cert-ACC: {}%'.format(cert_ACC))

Eps: 1
Samp-wise Cert-ACC: 39%
Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-24
Set parameter TimeLimit to value 600
UP-based Cert-ACC: 41.0%
